# Vamos aprender a trabalhar com PDF usando o Python

- Regra geral: PDF foi feito justamente para bloquear muita coisa, então não é fácil "brincar" com um pdf
- Mesmo assim, Python tem várias bibliotecas que vão nos ajudar, vamos focar em 2:
    - PyPDF2
    - Tabula
- Ler e extrair informações de um PDF a gente consegue fazer.
- Escrever e Editar, aí já é outra história

### Para os nossos exemplos, vamos avaliar o Release de Resultados do 3º e 4º Trimestre de 2020 da Magazine Luiza

In [1]:
%pip install pypdf2
%pip install tabula-py

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.0 MB 4.8 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/12.0 MB 5.2 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/12.0 MB 4.3 MB/s eta 0:00:03
   ------------- -------------------------- 3.9/12.0 MB 5.2 MB/s eta 0:00:02
   ------------------- -------------------- 5.8/12.0 MB 6.0 MB/s eta 0:00:02
   ------------------------- -------------- 7.6/12.0 MB 6.4 MB/s eta 0:00:01
   ------------------------------- -------- 9.4/12.0 MB 6.8 MB/s eta 0:00:01
   -------------------------------------- - 11.5/12.0 MB 7.2 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 7.3 MB/s  0:00:01

   -------------------- ------------------- 1/2 [tabula-py]
   ---------------------------------------- 2/2 [tabula-py]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 1º Objetivo: Queremos conseguir separar apenas o DRE do Release de Resultados (Página 14) para enviar para a Diretoria, como fazemos?
    - Separar as páginas de um pdf

In [6]:
import PyPDF2 as pyf
from pathlib import Path

nome = 'MGLU_ER_3T20_POR.pdf'
arquivo_pdf = pyf.PdfReader(nome)

#print(len(arquivo_pdf.pages))

for i, pagina in enumerate(arquivo_pdf.pages):
    num = i + 1
    #print(num)
    novo_pdf = pyf.PdfWriter()
    novo_pdf.add_page(pagina)
    with Path(f"paginas/Pagina {num}.pdf").open(mode="wb") as arquivo:
        novo_pdf.write(arquivo)

#### 2º Objetivo: Com o Release de Resultados já separado página por página, queremos incluir apenas as Páginas de Destaque (Página 1), DRE (Página 14) e Balanço (Página 16).
    - Juntar vários pdfs em 1

In [10]:
import PyPDF2 as pyf
from pathlib import Path

num_paginas = [1, 14, 16]

novo_arquivo = pyf.PdfWriter()

for num in num_paginas:
    pagina_pdf = pyf.PdfReader(f"paginas/Pagina {num}.pdf")
    novo_arquivo.add_page(pagina_pdf.pages[0])

with Path(f"Consolidado.pdf").open(mode='wb') as arquivo:
    novo_arquivo.write(arquivo)

### Extra: Para adicionar todas as páginas de 2 pdfs

In [9]:
pdf_mesclado = pyf.PdfMerger()

pdf_mesclado.append('MGLU_ER_3T20_POR.pdf')
pdf_mesclado.append('MGLU_ER_4T20_POR.pdf')

with Path(f"Mesclado.pdf").open(mode='wb') as arquivo:
    novo_arquivo.write(arquivo)

# Funcionalidades que podem ser úteis:

- Inserir arquivo no meio do outro
- Quero colocar dentro do Resultado do 4T20 os destaques do 3T20 para poder comparar os 2 dentro do mesmo relatório

In [3]:
import PyPDF2 as pyf
from pathlib import Path

pdf_mesclado = pyf.PdfMerger()

pdf_mesclado.append('MGLU_ER_4T20_POR.pdf')
pdf_mesclado.merge(1, 'paginas/Pagina 1.pdf')

with Path(f"Relatorio 2 Trimestres.pdf").open(mode='wb') as arquivo:
    pdf_mesclado.write(arquivo)


- Rodar Página

In [11]:
arquivo_pdf_original = pyf.PdfMerger('MGLU_ER_3T20_POR.pdf')

novo_arquivo = pyf.PdfWriter()

for pagina in arquivo_pdf_original.pages:
    pagina.rotate(90)
    novo_arquivo.add_page(pagina)

with Path(f"Relatorio Rotacionado.pdf").open(mode='wb') as arquivo:
    novo_arquivo.write(arquivo)

# Trabalhando com Textos e Informações Dentro do PDF

#### 1º Objetivo: Quero identificar como foram as Despesas com Vendas da MGLU
    - Pegar texto da página e identificar onde está essa informação

In [14]:
arquivo = pyf.PdfReader('MGLU_ER_3T20_POR.pdf')

qtd_paginas = len(arquivo.pages)
print(qtd_paginas)

metadados_arquivo = arquivo.metadata
print(metadados_arquivo)

24
{'/Title': 'DESEMPENHO FINANCEIRO CONSOLIDADO', '/Author': 'an_rezende', '/Subject': 'Receita Bruta', '/Creator': 'Microsoft® Office Word 2007', '/CreationDate': "D:20201109183121-03'00'", '/ModDate': "D:20201109183121-03'00'", '/Producer': 'Microsoft® Office Word 2007'}


In [18]:
texto_referencia = "| Despesas com Vendas"

for i, pagina in enumerate(arquivo.pages):
    texto_pagina = pagina.extract_text()
    if texto_referencia in texto_pagina:
        print('Numero pagina: ', i+1)
        texto_analisar = texto_pagina

Numero pagina:  10


In [ ]:
print(texto_analisar)

In [23]:
posicao_inicial = texto_analisar.find(texto_referencia)
posicao_final = texto_analisar.find("|", posicao_inicial + 1)

texto_final = texto_analisar[posicao_inicial:posicao_final]

print(texto_final)

| Despesas com Vendas  
 
No 3T20, as despesas com vendas totalizaram R$1.432,6 milhões, equivalentes a 17,2% da receita líquida, 1,1 p.p. menor que no 
3T19 , principalmente devido ao forte crescimento das vendas . Vale ressaltar que a Companhia conseguiu diluir as despesas com 
vendas m esmo investi ndo em maior nível de serviço,  especialmente em  atendimento e logística.  
 
Nos 9M20, as despesas com vendas totalizaram R$3.487,2 milhões, equivalentes a 18,2% da receita líquida (+1,1 p.p. versus  os 
9M19).  
 



#### 2º Objetivo: Quero analisar o DRE (sem ajuste - Página 5)
    - Para ler tabelas em pdf, use o tabula (é ninja)
    
    - Cuidado 1: Instale o tabula-py (não instale o tabula). Se instalar o tabula errado, desinstale ele, instale o tabula-py, desinstale o tabula-py e instale novamente o tabula-py. Reinicie o kernel do Jupyter após isso
    
    - Cuidado 2: Tem que ter o java instalado no seu computador (depois de instalar, reinicie o computador)

#### 3º Objetivo: Quero analisar o Capital de Giro e os Investimentos (ambas as tabelas na página 12)
    - Páginas com mais de 1 tabela

#### O que fazer quando o tabula não consegue ler alguma linha da tabela? Como o cabeçalho, no nosso caso?

# Outro método que pode ser útil algum dia: Captar Imagem em um pdf
    - biblioteca pikepdf

# Substituir texto no pdf tipo contrato

- Não recomendo fazer diretamente pelo Python. Realmente do que vi a melhor opção me parece o Word fazer isso
- Caso precise automatizar, automatize o processo fazendo ele pelo Word
- Quem quiser MUITO fazer isso pelo Python, tem um link aqui que vai te ajudar de uma solução que achei que funciona. Tem seus bugs/cuidados especiais, mas funciona: https://pdf.co/samples/pdf-co-web-api-replace-text-from-pdf-python-replace-text-from-url